In [ ]:
import sys
import os
dir = os.path.abspath('')
while not dir.endswith('ardt'): dir = os.path.dirname(dir)
if not dir in sys.path: sys.path.append(dir)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from datasets import load_dataset, load_from_disk
from huggingface_hub import login

from utils.helpers import find_root_dir

from access_tokens import HF_WRITE_TOKEN

In [ ]:
login(token=HF_WRITE_TOKEN)
ARDT_DIR = find_root_dir()
DIR = ARDT_DIR + "/datasets-raw/nonadv-raw-data/"
ENV_NAME = "hopper"

In [32]:
def compute_sum(ds):
    return {'returns': sum(ds['rewards'])}

filenames = os.listdir(DIR)
for filename in filenames:
    if ENV_NAME in filename:
        filepath = DIR + filename
        dataset = load_from_disk(filepath)

        print(f"File {filename} has {len(dataset['actions'])} episodes.")

        total_number_of_steps = 0
        for l in dataset['actions']:
            total_number_of_steps += len(l)

        print("Confirming total episodes: ", len(dataset['actions']))
        print("Total steps: ", total_number_of_steps)

        pr_actions = dataset['actions']
        adv_actions = [[0.0] * len(sublist) for sublist in pr_actions]

        dataset = dataset.add_column('pr_actions', pr_actions)
        dataset = dataset.add_column('adv_actions', adv_actions)
        dataset = dataset.remove_columns(['actions'])

        print("Dataset info:\n", dataset)

        print("Plotting returns distribution...")
        ds_to_vis = dataset.map(compute_sum)
        sns.displot(ds_to_vis['returns'], kind="kde", bw_adjust=0.5)
        plt.title(f"Returns distribution for {filename}")
        plt.show()

        print("=================================================================")



In [ ]:
# TYPE = 'train'
# VERSION = 'v4'

# dataset.save_to_disk(f'{ARDT_DIR}/datasets/ppo_noadv_{TYPE}_halfcheetah_{VERSION}')